In [1]:
import tensorflow
from util import PATH_FOR_OUR_TRAINING_DATA, UPPER_LIMIT_OF_IMAGES, TARGET_SHAPE
import pandas as pd
from DS_Generator import DS_Generator
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from util import PATH_FOR_OUR_TRAINING_DATA

In [5]:
df = pd.read_csv(PATH_FOR_OUR_TRAINING_DATA + "/data.csv", index_col=0)
df = df[df["species_counts"]>100].sort_values(["species_counts"])
ds = DS_Generator().generate(df)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=TARGET_SHAPE+(3,)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())


In [7]:
model.fit(
    ds,
    epochs=5)

Epoch 1/5
393/393 [==============================] - 80s 196ms/step - loss: 1.0054
Epoch 2/5
393/393 [==============================] - 77s 197ms/step - loss: 1.0020
Epoch 3/5
393/393 [==============================] - 80s 202ms/step - loss: 1.0018
Epoch 4/5
393/393 [==============================] - 81s 206ms/step - loss: 1.0017
Epoch 5/5
393/393 [==============================] - 79s 202ms/step - loss: 1.0016
